# Visualize with Plotly

In [2]:
import csv

import sqlite3
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import plotly.offline as offline



filepath = "db.sqlite3"
conn = sqlite3.connect(filepath)
cur = conn.cursor()

In [18]:
# list up slides 
cur.execute(f"""
    SELECT slide_id, slide_title 
    FROM slide;
""")

In [19]:

slide_id_to_title = {}
for slide_id, slide_title in cur.fetchall():
    # print(slide_id, slide_title)
    slide_id_to_title[slide_id] = slide_title

# 中身が入っていることを確認
assert slide_id_to_title != {}

In [3]:
# 1つに対して
table_name = 'slide_pv_date'

slide_id = 'b9cb90f9e11845c9afe619a61c64798b'

cur.execute(f"""
    select pv, date from slide_pv_date WHERE slide_id="{slide_id}" ORDER BY date;
""")


In [4]:
date_list = []
pv_list = []
for pv, date in cur.fetchall():
    date_list.append(datetime.strptime(date, "%Y-%m-%d"))
    pv_list.append(pv)


In [5]:
trace0 = go.Scatter(x = date_list, y = pv_list, mode = 'lines', name = 'X')


# レイアウトの指定
layout = go.Layout(xaxis = dict(title = 'date', type='date'), 
              yaxis = dict(title = 'pv'))

fig = dict(data = [trace0], layout = layout)

offline.iplot(fig)

In [27]:
slide_id_list = [id_ for id_ in slide_id_to_title.keys()]
lines = []
for slide_id in slide_id_list:
    cur.execute(f"""
        select pv, date from slide_pv_date WHERE slide_id="{slide_id}" ORDER BY date;
    """)

    records = [[pv, date] for pv, date in cur.fetchall()]
    df = pd.DataFrame(records, columns =["pv", "date"])

    lines.append(
        go.Scatter(x=df['date'], y=df['pv'], mode = 'lines', name = slide_id_to_title[slide_id])
        )

fig = go.Figure(lines)

offline.iplot(fig)

# trace1 = go.Scatter(x = date_list, y = pv_list, mode = 'lines', name = 'X')


In [51]:
fig = go.Figure([go.Scatter(x=df['date'], y=df['pv'])])

offline.iplot(fig)


In [ ]:
# コネクションをクローズする
conn.close()